# Lab | Avila Bible 

In this lab, we will explore the [**Avila Bible dataset**](https://archive.ics.uci.edu/ml/datasets/Avila) which has been extracted from 800 images of the 'Avila Bible', an XII century giant Latin copy of the Bible. The prediction task consists in associating each pattern to a copyist. You will use supervised learning algorithms to figure out what feature patterns each copyist are likely to have and use our model to predict those copyist.

-----------------------------------------------------------------------------------------------------------------

## Before your start:
    - Read the README.md file,
    - Comment as much as you can and use the APIla-bible in the README.md,
    - Happy learning!

In [1]:
# Import your libraries
import pandas as pd
import numpy as np
import requests
from scipy import stats
import sklearn
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

![machine-learning](https://miro.medium.com/proxy/1*halC1X4ydv_3yHYxKqvrwg.gif)

The Avila data set has been extracted from 800 images of the the **Avila Bible**, a giant Latin copy of the whole Bible produced during the XII century between Italy and Spain. The palaeographic analysis of the  manuscript has  individuated the presence of 12 copyists. The pages written by each copyist are not equally numerous. 
Each pattern contains 10 features and corresponds to a group of 4 consecutive rows.

## What am I expected to do?

Well, your prediction task consists in associating each pattern to one of the 8 monks we will be evaluating (labeled as:  Marcus, Clarius, Philippus, Coronavirucus, Mongucus, Paithonius, Ubuntius, Esequlius). For that aim, you should: 
- Train a minimum of 4 different models
- Perform a minimum of 4 Feature Extraction and Engineering techniques
- Must contain a summary of the machine learning tools and algorithms
- and the results or the score obtained with each of them

You won't get much more instructions from now on. Remember to comment your code as much as you can. Keep the requirements in mind and have fun! 

Just one last piece of advice, take a moment to explore the data, remember this dataset contains two files: **train** and **test**. You will find both files in `data` folder. The **test** files contains the data you will predict for, therefore it does not include the labels.
Use the **train** dataset as you wish, but don't forget to split it into **train** and **test** again so you can evaluate your models. Just be sure to train it again with the whole data before predicting.
We have also included a **sample submission** which is of the exact shape and format you must use when evaluating your predictions against the groundtruth through the `APIla-bible`. It won't work unless it is the exact same shape. 



#### Train dataset

In [2]:
train_dataset = pd.read_csv('../data/training_dataset.csv', index_col=0)

In [3]:
train_dataset.head()

,0,1,2,3,4,5,6,7,8,9,10
0,0.241386,0.109171,-0.127126,0.380626,0.172340,0.314889,0.484429,0.316412,0.188810,0.134922,Marcus
1,0.303106,0.352558,0.082701,0.703981,0.261718,-0.391033,0.408929,1.045014,0.282354,-0.448209,Clarius
2,-0.116585,0.281897,0.175168,-0.152490,0.261718,-0.889332,0.371178,-0.024328,0.905984,-0.877830,Philippus
3,-0.326430,-0.652394,0.384996,-1.694222,-0.185173,-1.138481,-0.232828,-1.747116,-1.183175,-0.807380,Philippus
4,-0.437525,-0.471816,0.463236,-0.545248,0.261718,-0.972381,0.824183,-3.108388,-2.991700,-1.141030,Philippus


#### Test dataset


In [4]:
test_dataset = pd.read_csv('../data/test_dataset.csv', index_col=0)

In [5]:
test_dataset.head()

,0,1,2,3,4,5,6,7,8,9
0,-0.017834,0.132725,0.125378,1.357345,0.261718,0.190314,0.182426,0.445253,-0.715453,0.189796
1,-0.202992,-0.000745,-3.210528,-0.527256,0.082961,0.771662,0.144676,0.098572,0.251173,0.745333
2,1.019049,0.211237,-0.155578,-0.311855,0.261718,0.107265,0.484429,0.339303,-0.310094,-0.049630
3,0.451232,-0.267686,0.335206,-0.831336,0.261718,0.024215,0.220177,0.988787,0.032902,0.025485
4,-0.227680,0.109171,0.413447,0.118917,0.172340,0.480988,0.522180,0.091562,0.313536,0.256389


#### Sample submission

In [6]:
sample_submission = pd.read_csv('../data/sample_submission.csv', header=None, index_col=0)

In [7]:
sample_submission.head()

,1
0,
0,Philippus
1,Ubuntius
2,Esequlius
3,Coronavirucus
4,Philippus


`Keep calm and code on!`

# Challenge - train your models, make the best prediction

### Preprocesado de los datos

In [8]:
# Definir la X, variables predictoras
X = train_dataset.drop(["10"], axis=1)
# Definir la variable dependiente
y = train_dataset[["10"]].rename(columns={"10": "autor"})
# Valores únicos
y["autor"].unique()
# Asignar un número de clase a los autores, por si el clasificador no admite nombres
y.loc[(y.autor == 'Marcus'),'class']='1'
y.loc[(y.autor == 'Clarius'),'class']='2'
y.loc[(y.autor == 'Philippus'),'class']='3'
y.loc[(y.autor == 'Coronavirucus'),'class']='4'
y.loc[(y.autor == 'Mongucus'),'class']='5'
y.loc[(y.autor == 'Paithonius'),'class']='6'
y.loc[(y.autor == 'Ubuntius'),'class']='7'
y.loc[(y.autor == 'Esequlius'),'class']='8'

#### Comprobar si hay datos nulos

In [10]:
X.isna().sum().sort_values(ascending=False)

9    0
8    0
7    0
6    0
5    0
4    0
3    0
2    0
1    0
0    0
dtype: int64

In [11]:
# Estadística básica descriptiva de los datos para ver qué tenemos
X.describe()

,0,1,2,3,4,5,6,7,8,9
count,12017.000000,12017.000000,12017.000000,12017.000000,12017.000000,12017.000000,12017.000000,12017.000000,12017.000000,12017.000000
mean,0.021280,0.030684,-0.000379,-0.022127,0.006801,-0.001279,0.032540,-0.006720,-0.011368,-0.026942
std,1.004481,3.661030,1.072319,1.002045,0.963679,1.108192,1.245215,1.012995,1.085821,0.985799
min,-3.498799,-2.426761,-3.210528,-5.440122,-4.922215,-7.450257,-11.935457,-4.164819,-5.486218,-6.719324
25%,-0.128929,-0.259834,0.064919,-0.542563,0.172340,-0.598658,-0.006326,-0.555747,-0.372457,-0.528135
50%,0.056229,-0.055704,0.214288,0.080127,0.261718,-0.058835,0.220177,0.101115,0.064084,-0.053548
75%,0.216699,0.203385,0.349432,0.601905,0.261718,0.522513,0.446679,0.646377,0.500624,0.491862
max,11.819916,386.000000,50.000000,3.987152,1.066121,53.000000,83.000000,13.173081,44.000000,11.911338


#### Filtrado de outliers

In [ ]:
X_cl = X[(np.abs(stats.zscore(X)) < 3).all(axis=1)]
X_cl.describe()

In [ ]:
from sklearn.decomposition import PCA
# Definir la PCA
pca = PCA(n_components=2)
# Ajustar a los datos
X_pca = pca.fit_transform(X_cl)
# Convertir a dataframe
PCA_df = pd.DataFrame(data = X_pca, columns = ['PC1', 'PC2'])
# Añadir las equiquetas
PCA_df = pd.concat([PCA_df, y['autor']], axis = 1)

PCA_df.head()

In [ ]:
figure(num=None, figsize=(8, 8), dpi=80, facecolor='w', edgecolor='k')

sns.scatterplot(PCA_df.PC1, PCA_df.PC2, hue = PCA_df["autor"])    
plt.xlabel('Principal Component 1', fontsize = 12)
plt.ylabel('Principal Component 2', fontsize = 12)
plt.title('2D PCA', fontsize = 15)
plt.legend(PCA_df["autor"])
plt.grid()

In [ ]:
# Ajustar a los datos
X_pca = pca.fit_transform(X)
# Convertir a dataframe
PCA_df = pd.DataFrame(data = X_pca, columns = ['PC1', 'PC2'])
# Añadir las equiquetas
PCA_df = pd.concat([PCA_df, y['autor']], axis = 1)

PCA_df.head()

In [ ]:
figure(num=None, figsize=(8, 8), dpi=80, facecolor='w', edgecolor='k')

sns.scatterplot(PCA_df.PC1, PCA_df.PC2, hue = PCA_df["autor"])    
plt.xlabel('Principal Component 1', fontsize = 12)
plt.ylabel('Principal Component 2', fontsize = 12)
plt.title('2D PCA', fontsize = 15)
plt.legend(PCA_df["autor"])
plt.grid()

In [ ]:
MODELOS

Regresión Logística datos en bruto

In [12]:
# Dividir el 75% de datos para muestras de entrenamiento y el 25% para validación
X_train, X_test, y_train, y_test = train_test_split(X, y["autor"], test_size=0.25)

# Definir y aplicar el modelo
model_bruto =  LogisticRegression(solver="newton-cg").fit(X_train, y_train)
# Hacer la predicción
y_test_pred = model_bruto.predict(X_train)

#comparison = pd.DataFrame({"y_test_pred":y_test_pred,"gt":y_test})
#comparison

print(classification_report(y_test_pred,y_train))

               precision    recall  f1-score   support

      Clarius       0.10      0.39      0.16       450
Coronavirucus       0.87      0.87      0.87       775
    Esequlius       0.00      0.00      0.00         4
       Marcus       0.94      0.54      0.68      6676
     Mongucus       0.73      0.78      0.75       418
   Paithonius       0.18      0.51      0.26       159
    Philippus       0.27      0.51      0.35       529
     Ubuntius       0.00      0.00      0.00         1

     accuracy                           0.57      9012
    macro avg       0.38      0.45      0.38      9012
 weighted avg       0.83      0.57      0.65      9012



Calcular el modelo para todo el train_set


In [13]:
# Definir y aplicar el modelo
model_bruto_todo =  LogisticRegression(solver="newton-cg").fit(X, y["autor"])


## What do I do once I have a prediction?

Once you have already trained your model and made a prediction with it, you are ready to check what is the accuracy of it. 

Save your prediction as a `.csv` file.

In [15]:
# Hacer la predicción con el test set
y_pred_todo = model_bruto_todo.predict(test_dataset)

sub = pd.DataFrame(y_pred_todo).to_csv(header=None)

Now you are ready to know the truth! Are you good enough to call yourself a pro?

Lucky you have the ultimate **APIla-bible** which give you the chance of checking the accuracy of your predictions as many times as you need in order to become the pro you want to be. 

## How do I post my prediction to the APIla-bible?

Easy peasy! You should only fulfil the path to your prediction `.csv` and run the cell below! 

In [16]:
my_submission = "../data/sample_submission.csv"
with open(my_submission) as f:
    res = requests.post("http://apila-bible.herokuapp.com/check", files={"csv_data":sub})
res.json()

KeyboardInterrupt: 

![hope-you-enjoy](https://imgs.xkcd.com/comics/machine_learning.png)